In [ ]:
import mealpy
import numpy as np

from sim.sim_controllers.polyfit_controller import *
from sim.config import *

In [ ]:
from pprint import pprint

################################ User Input ################################

time_config_file = None
exp_config_file = None

############################################################################

time_config = TimingConfig.load_json(time_config_file)
exp_config = ExperimentConfig.load_json(exp_config_file)

pprint(time_config)

In [ ]:
poly_degree = 1

input_offsets = np.asanyarray(
    [
        -2 * time_config.cycle_frame_num + 0,
        -2 * time_config.cycle_frame_num + 5,
        -2 * time_config.cycle_frame_num + 9,
        -time_config.cycle_frame_num + 0,
        -time_config.cycle_frame_num + 5,
        -time_config.cycle_frame_num + 9,
        0,
        5,
        9,
    ]
)

In [ ]:
start_times = np.arange(exp_config.num_frames // time_config.cycle_frame_num) * time_config.cycle_frame_num

evaluator = WeightEvaluator(
    "bboxes.csv",
    time_config,
    input_offsets=input_offsets,
    start_times=start_times,
    eval_offset=time_config.cycle_frame_num + time_config.imaging_frame_num // 2,
)


def eval_func(weights: np.ndarray) -> float:
    return evaluator.eval(weights, deg=poly_degree)

In [ ]:
from mealpy.swarm_based.PSO import OriginalPSO
from mealpy.utils.problem import Problem
from mealpy.utils.termination import Termination
from mealpy.utils.agent import Agent
import mealpy

# optim = mealpy.ICA.OriginalICA()
optim = mealpy.PSO.OriginalPSO()

termination = Termination(
    max_epoch=300,
    max_fe=None,
    max_time=None,
    max_early_stop=100,
)

bounds = mealpy.FloatVar(lb=np.zeros(len(input_offsets)), ub=np.ones(len(input_offsets)))

problem = Problem(obj_func=eval_func, bounds=bounds, minimax="min")

In [ ]:
best: Agent = optim.solve(problem, termination=termination)

In [ ]:
np.set_printoptions(precision=4, suppress=True)
print(optim.g_best.target.fitness)
print(optim.g_best.solution / np.linalg.norm(best.solution))

In [ ]:
poly_config = PolyfitConfig(
    degree=poly_degree,
    sample_times=evaluator.input_offsets.tolist(),
    weights=optim.g_best.solution.tolist(),
)

poly_config.save_json("polyfit_config.json")